# Big Data Project: Transforming Scientific Articles into Videos with Speech using Apache Spark and Kafka


In [1]:
# make modules from py files auto-reload when changed
%load_ext autoreload
%autoreload 2

#TODO:   
create folders for data, checkpoints, output, dbg


# TTS 


## Basic run just for test 

In [2]:

from datetime import datetime
from ArticleReader.Chunker import Chunker
from ArticleReader.LatexToSpeech import LatexParser
from ArticleReader.Narrator import Narrator


/home/linuxu/anaconda3/envs/pyspark_TTS/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
INFO:speechbrain.utils.quirks:Applied quirks (see `speechbrain.utils.quirks`): [disable_jit_profiling, allow_tf32]
INFO:speechbrain.utils.quirks:Excluded quirks specified by the `SB_DISABLE_QUIRKS` environment (comma-separated list): []
/home/linuxu/anaconda3/envs/pyspark_TTS/lib/python3.9/site-packages/IPython/extensions/autoreload.py:211: UserWarning: Module 'speechbrain.pretrained' was deprecated, redirecting to 'speechbrain.inference'. Please update your script. This is a change from SpeechBrain 1.0. See: https://github.com/speechbrain/speechbrain/releases/tag/v1.0.0
  if not hasattr(module, "__file__") or module.__file__ is None:


In [3]:
input_file = "data/arXiv-2106.04624v1/main.tex"
output_file = "output/" + datetime.now().strftime(r"%y.%m.%d-%H")

parser = LatexParser()
content = parser.read_latex(input_file)
processed = parser.custom_latex_to_text(content)
parser.save_text(processed, "dbg/spec_my.txt")

tables = parser.get_tables()
parser.save_text(tables, "dbg/tables.tex")

In [ ]:
chunker = Chunker(max_len=200)
chunker.split_text_into_chunks(processed)
n_chunks = chunker.get_test_batch(10, 0)
# chunks = chunker.chunks
chunker.save_chunks_as_text(output_file + ".md", n_chunks)

,index,sentence,text_len
0,0,si plus plus,12
1,1,\n\n,4
2,2,\n \n \n \n \n \n \n \n \n \n \n \n \n \n \n ...,40
3,3,\n\n,4
4,4,SpeechBrain: A General-Purpose Speech Toolkit.,46
5,5,\n\n,4
6,6,SpeechBrain is an open-source and all-in-one s...,61
7,7,It is designed to facilitate the research and ...,162
8,8,\nThis paper describes the core architecture d...,189
9,9,SpeechBrain achieves competitive or state-of-t...,103


# Spark 

In [11]:
#import findspark
import os
import sys
import pandas as pd 
import torch

from pyspark.sql import SparkSession, Window
from pyspark.sql.functions import pandas_udf, PandasUDFType, udf, col, lit, desc, floor
from pyspark.sql.functions import collect_list, flatten
from pyspark.sql import functions as F
from pyspark.sql.types import *
from pyspark import pandas as psp


/home/linuxu/anaconda3/envs/pyspark_TTS/lib/python3.9/site-packages/pyspark/pandas/__init__.py:50: UserWarning: 'PYARROW_IGNORE_TIMEZONE' environment variable was not set. It is required to set this environment variable to '1' in both driver and executor sides if you use pyarrow>=2.0.0. pandas-on-Spark will set it for you but it does not work if there is a Spark context already launched.
  warnings.warn(


In [12]:
sys.executable

'/home/linuxu/anaconda3/envs/pyspark_TTS/bin/python'

In [13]:
os.getcwd()

'/home/linuxu/Michael/BigData/ArticleReader'

In [14]:
os.environ['PYTHONPATH'] = '/usr/local/spark/spark/python/lib/py4j-0.10.9.7-src.zip:/usr/local/spark/spark/python/:'

In [15]:
# Set Spark environment variables
print(os.environ['SPARK_HOME'] )
print(os.environ['PYSPARK_PYTHON'])
print(os.environ['PYSPARK_DRIVER_PYTHON'] )
print(os.environ['PYTHONPATH'] )
#print(os.environ['JAVA_HOME'] )

/usr/local/spark/spark
/home/linuxu/anaconda3/envs/pyspark_TTS/bin/python
ipython
/usr/local/spark/spark/python/lib/py4j-0.10.9.7-src.zip:/usr/local/spark/spark/python/:


In [16]:
os.environ

environ{'CHROME_DESKTOP': 'code-url-handler.desktop',
        'DBUS_SESSION_BUS_ADDRESS': 'unix:path=/run/user/1000/bus,guid=99a19c09a7fc26df45954dbf67cae258',
        'DBUS_STARTER_ADDRESS': 'unix:path=/run/user/1000/bus,guid=99a19c09a7fc26df45954dbf67cae258',
        'DBUS_STARTER_BUS_TYPE': 'session',
        'DESKTOP_SESSION': 'ubuntu',
        'DISPLAY': ':100',
        'GDK_BACKEND': 'x11',
        'GDMSESSION': 'ubuntu',
        'GIO_LAUNCHED_DESKTOP_FILE': '/usr/share/applications/code.desktop',
        'GIO_LAUNCHED_DESKTOP_FILE_PID': '14923',
        'GNOME_DESKTOP_SESSION_ID': 'this-is-deprecated',
        'GNOME_SHELL_SESSION_MODE': 'ubuntu',
        'GPG_AGENT_INFO': '/run/user/1000/gnupg/S.gpg-agent:0:1',
        'GTK_IM_MODULE': 'ibus',
        'GTK_MODULES': 'gail:atk-bridge',
        'HOME': '/home/linuxu',
        'INVOCATION_ID': 'a336489572c445eaad1bd3de0cfdb8fd',
        'JOURNAL_STREAM': '8:145702',
        'LANG': 'en_IL',
        'LANGUAGE': 'en_IL:en',
        

In [17]:

# Get number of CPU cores
# workers = 1
# cpus_limit = os.cpu_count() -1 
# mem_limit = "1g" # prod: "16g"

# simulating a cluster with 2 workers
workers = 2
cpus_limit =  int(os.cpu_count()/ workers) -1 
mem_limit = "2g" # prod: "16g"/ workers



# Configure PyTorch for CPU parallelism
torch.set_num_threads(cpus_limit)

# Create Spark session with configurations
spark = SparkSession.builder \
    .appName("TTS CPU Inference") \
    .master("local[*]") \
    .config("spark.executor.cores", cpus_limit) \
    .config("spark.executor.instances", workers) \
    .config("spark.executor.memory", mem_limit) \
    .config("spark.task.cpus", cpus_limit) \
    .config("spark.dynamicAllocation.enabled", "false") \
    .config("spark.sql.shuffle.partitions", "1") \
    .getOrCreate()

    # Additional configs that might be useful in future:
    # .config("spark.executor.resource.gpu.amount", "1") \
    # .config("spark.executor.memoryOverhead", "<memory>"

print(f"Spark version: {spark.version}")
print(f"Using {cpus_limit} CPU cores for inference.")

spark.sparkContext

25/03/13 18:17:54 WARN Utils: Your hostname, Ubuntu-GPU-63 resolves to a loopback address: 127.0.1.1; using 192.168.90.110 instead (on interface ens160)
25/03/13 18:17:54 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
25/03/13 18:17:55 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


Spark version: 3.5.4
Using 7 CPU cores for inference.


<SparkContext master=local[*] appName=TTS CPU Inference>

1. Memory Allocation and Executor Configuration

    Spark executors are allocated a fixed amount of memory when the application starts. This memory is split between:
        JVM heap memory: For Spark tasks and operations.
        Off-heap memory: For operations like shuffle or caching.
        GPU memory (if applicable): Used by PyTorch or other frameworks.
    Memory allocation is configured using:

--executor-memory <memory>
--driver-memory <memory>
--conf spark.memory.fraction=<fraction>

2. Dynamic Resource Allocation

    Spark can adjust resource allocation dynamically if dynamic resource allocation is enabled.
    Configuration:

```
--conf spark.dynamicAllocation.enabled=true
--conf spark.dynamicAllocation.minExecutors=<min>
--conf spark.dynamicAllocation.maxExecutors=<max>
```
3. Task Partitioning and Locality

    Spark divides the dataset into partitions, which determine the unit of work assigned to each executor.
    Smaller partitions reduce the likelihood of memory overload but increase overhead. Aim for ~128MB partition sizes for large datasets.
    Control partition size with:
```
--conf spark.sql.files.maxPartitionBytes=<size>
```

## Playground for batching

In [ ]:
sentences

In [ ]:
waveforms

In [ ]:
mel_lens = narrator.add_pauses(sentences.sentence, mel_lengths, pause_dur=40)   

In [ ]:
mel_lengths

In [ ]:
mel_lens

In [ ]:
arr = torch.tensor_split(waveforms.squeeze(1), len(waveforms), dim=0)

#arr = [a[:, :l] for a, l in zip(arr, mel_lengths * self.hop_len)]

# TODO: cut padding 
arr = [a[:, :l].squeeze(0).numpy() for a, l in zip(arr, mel_lens * narrator.hop_len)]


In [ ]:
arr[0].shape

In [ ]:
type(arr[0])

In [ ]:
arr[1].shape

## Pandas UDF approach 


In [173]:

@pandas_udf(StructType([StructField("waveform", ArrayType(FloatType())),StructField("mel_lengths", IntegerType())]))
def predict_batch_udf(sentences: pd.Series) -> pd.DataFrame:
  # TODO: calculate and store "seq_len"
  # TODO: non-default model initialization
  narrator = Narrator()    
  waveforms, mel_lengths = narrator.infer(sentences)

  arr = torch.tensor_split(waveforms.squeeze(1), len(waveforms), dim=0)

  # Add more pause where needed (very naive currenty)
  mel_lengths = narrator.add_pauses(sentences, mel_lengths, pause_dur=40)   
  # Cut silence padding while applying pauses from above 
  arr = [a[:, :l].squeeze(0).numpy() for a, l in zip(arr, mel_lengths * narrator.hop_len)]  
  
  output = pd.DataFrame({"waveform": arr, "mel_lengths": mel_lengths})
 
  return output


In [20]:

sc = spark.sparkContext

sc

<SparkContext master=local[*] appName=TTS CPU Inference>

In [66]:

input_file = "data/arXiv-2106.04624v1/main.tex"


In [67]:
def preprocess_text(file_content):
    parser = LatexParser()
    processed_text = parser.custom_latex_to_text(file_content)   

    return processed_text, parser.tables, parser.figures
    # return { "text": processed_text, 
    #         "tables": parser.tables, 
    #         "figures": parser.figures } 



In [68]:
df = sc.wholeTextFiles(input_file)

processed_rdd = df.mapValues(preprocess_text)

In [75]:
# Convert RDD to DataFrame
from pyspark.sql import Row

df_processed = processed_rdd.map(lambda x: Row(filename=x[0], 
                                               text=x[1][0], 
                                               tables=x[1][1], 
                                               figures=x[1][2])).toDF()

In [80]:
# Define the schema matching the Pandas DataFrame structure
chunk_schema = StructType([
    StructField("index", IntegerType(), False),  
    StructField("sentence", StringType(), False),  
    StructField("text_len", IntegerType(), False)
])

# Register as a Pandas UDF


In [ ]:
#@pandas_udf(chunk_schema)

@udf(ArrayType(StringType()))
def udf_split_text(text, chunk_size=500, test =False):
    from ArticleReader.Chunker import Chunker
    chunker = Chunker(max_len=chunk_size)
    chunker.split_text_into_chunks(text)
    return chunker.get_chunks()
    

df_chunks = df_processed.withColumn("chunks", udf_split_text(df_processed["text"]))


In [158]:
df_final = df_chunks.selectExpr("filename", "explode(chunks) as sentence")


In [161]:
from pyspark.sql.functions import monotonically_increasing_id 

df2 = df_final.withColumn("index", monotonically_increasing_id()) \
    .selectExpr("*"," length(sentence) as text_len").offset(295).limit(15)

In [165]:
df2.selectExpr('count(sentence)').show()

+---------------+
|count(sentence)|
+---------------+
|             15|
+---------------+



In [166]:
# Apply the UDF
text_volume_window = (Window.orderBy(desc('text_len'))
             .rowsBetween(Window.unboundedPreceding, 0))
# TODO: maybe can use partitionng here for separating whole text into chapters? 

text_volume_max  = 600 

step1 = df2.withColumn('cum_text_volume', F.sum('text_len').over(text_volume_window)) \
.withColumn('part', floor(col('cum_text_volume')/lit(text_volume_max)) )
step1

DataFrame[filename: string, sentence: string, index: bigint, text_len: int, cum_text_volume: bigint, part: bigint]

In [167]:
nparts =  step1.select((lit(1) + F.max("part")).alias("npart")).first()
nparts

25/03/13 20:06:39 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
25/03/13 20:06:39 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
25/03/13 20:06:40 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
25/03/13 20:06:40 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.


Row(npart=3)

In [168]:
nparts[0]

3

In [169]:
step1.show()

25/03/13 20:06:44 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
25/03/13 20:06:44 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
25/03/13 20:06:44 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.


+--------------------+--------------------+-----+--------+---------------+----+
|            filename|            sentence|index|text_len|cum_text_volume|part|
+--------------------+--------------------+-----+--------+---------------+----+
|file:/home/linuxu...|In neuroscience, ...|  296|     398|            398|   0|
|file:/home/linuxu...|To make inference...|  300|     244|            642|   1|
|file:/home/linuxu...|In this case, we ...|  304|     218|            860|   1|
|file:/home/linuxu...|In the following,...|  308|     151|           1011|   1|
|file:/home/linuxu...|The inference API...|  302|     141|           1152|   1|
|file:/home/linuxu...|Subsection: Exper...|  306|      31|           1183|   1|
|file:/home/linuxu...|Subsubsection: In...|  298|      25|           1208|   2|
|file:/home/linuxu...|                \n\n|  295|       4|           1212|   2|
|file:/home/linuxu...|                \n\n|  297|       4|           1216|   2|
|file:/home/linuxu...|                \n

25/03/13 20:06:45 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
25/03/13 20:06:45 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.


In [170]:
step1 = step1.repartitionByRange(nparts[0], "part")

In [171]:
step1.rdd.getNumPartitions()

25/03/13 20:07:03 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
25/03/13 20:07:03 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
25/03/13 20:07:04 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
25/03/13 20:07:04 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.


3

In [174]:

step2 = step1.withColumn("prediction", predict_batch_udf(col("sentence"))).cache()


25/03/13 20:07:44 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
25/03/13 20:07:44 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.


In [175]:

step3 = step2.select("index", "sentence", "text_len", "prediction.*")

step4 = step3.sort("index")

print("recombine batch")
wf = step4.agg(flatten(collect_list(col("waveform")))).alias("speech")

#torch.cat(tuple(batch_converted.waveform), dim=1)

final = step4.cache()

recombine batch


In [176]:
step4.select(F.size("waveform")).show()

25/03/13 20:08:10 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
25/03/13 20:08:10 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
INFO:speechbrain.utils.quirks:Applied quirks (see `speechbrain.utils.quirks`): [allow_tf32, disable_jit_profiling]
INFO:speechbrain.utils.quirks:Excluded quirks specified by the `SB_DISABLE_QUIRKS` environment (comma-separated list): []
INFO:speechbrain.utils.quirks:Applied quirks (see `speechbrain.utils.quirks`): [allow_tf32, disable_jit_profiling]
INFO:speechbrain.utils.quirks:Excluded quirks specified by the `SB_DISABLE_QUIRKS` environment (comma-separated list): []
INFO:speechbrain.utils.fetching:Fetch hyperparams.yaml: Fetching from HuggingFace Hub 'speechbrain/tts-tacotron2-ljspeech' if not cached
INFO:speechbrain.utils.fetching:Fetch hyperparams.yaml: Fet

PythonException: 
  An exception was thrown from the Python worker. Please see the stack trace below.
Traceback (most recent call last):
  File "/tmp/ipykernel_387938/4270153668.py", line 5, in predict_batch_udf
  File "/home/linuxu/Michael/BigData/ArticleReader/ArticleReader/Narrator.py", line 10, in __init__
    self.loadModels(tts_model, vocoder_model)
  File "/home/linuxu/Michael/BigData/ArticleReader/ArticleReader/Narrator.py", line 19, in loadModels
    self.tts = Tacotron2.from_hparams(
  File "/home/linuxu/anaconda3/envs/pyspark_TTS/lib/python3.9/site-packages/speechbrain/inference/interfaces.py", line 472, in from_hparams
    hparams_local_path = fetch(
  File "/home/linuxu/anaconda3/envs/pyspark_TTS/lib/python3.9/site-packages/speechbrain/utils/fetching.py", line 398, in fetch
    return link_with_strategy(fetched_file, destination, local_strategy)
  File "/home/linuxu/anaconda3/envs/pyspark_TTS/lib/python3.9/site-packages/speechbrain/utils/fetching.py", line 162, in link_wit

INFO:speechbrain.utils.fetching:Fetch model.ckpt: Fetching from HuggingFace Hub 'speechbrain/tts-tacotron2-ljspeech' if not cached
INFO:speechbrain.utils.parameter_transfer:Loading pretrained files for: model
/home/linuxu/anaconda3/envs/pyspark_TTS/lib/python3.9/site-packages/speechbrain/utils/checkpoints.py:200: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you sta

In [ ]:
final.show()

In [ ]:
wfc = wf.cache()

In [ ]:
wfc.show()

In [ ]:
waveform = wfc.collect()[0]

In [ ]:
narrator = Narrator()

In [ ]:
tens = torch.Tensor(waveform)

In [ ]:
tens.shape

In [ ]:
case_file = "output/spark_test"


print("saving sound")
narrator.save_audio(case_file + ".wav",tens )
print("done saving sound")

In [ ]:
def calculate_batch_size(memory_per_sentence, total_memory, safety_factor=0.8):
    # safety_factor reserves a buffer (e.g., 80% of total memory)
    return int((total_memory * safety_factor) / memory_per_sentence)

batch_size = calculate_batch_size(memory_per_sentence=500_000_000, total_memory=16_000_000_000)
batch_size
# Result: batch_size = 25

## Regular UDF approach

In [ ]:

n_chunks = psp.from_pandas(chunker.get_test_batch(3,53))
#chunks = spark.from_pandas(chunker.chunks)

In [ ]:
type(n_chunks)

In [ ]:
n_chunks.head()

In [ ]:

# Estimate batch size
batch_size = calculate_batch_size(memory_per_sentence=500_000_000, total_memory=16_000_000_000)
batch_size


In [ ]:
n_chunks.count()

In [ ]:

# Repartition the DataFrame to match the batch size
num_partitions = max(1, n_chunks.sentence.count() // batch_size)
num_partitions


In [ ]:

# from pyspark import SparkContext
# import torch

# sc = SparkContext.getOrCreate()

# # Load your PyTorch model
# model = torch.load('path_to_model.pth', map_location='cpu')
# model.eval()

# Broadcast the model
#broadcast_model = sc.broadcast(model)



In [ ]:
# Assign the structure and naming for newly created columns
schema = StructType([StructField("waveforms",
                                 IntegerType(), False),
                     StructField("mel_lengths",
                                 IntegerType(), False)])

In [ ]:
@udf('string', PandasUDFType.SCALAR)
def tts_udf(batch):
    narrator = Narrator()    
    waveforms, mel_lengths = narrator.infer(batch)    
    return waveforms, mel_lengths


    # ensure sentences are sorted by seq_len
    batch_df.loc[:,"seq_len"] = batch_df.sentence.map(narrator.seq_len)
    batch_df.sort_values("seq_len", ascending=False, inplace=True)

    # # defining pauses between paragraphs                
    # mel_lengths = narrator.add_pauses(batch_df.sentence, mel_lengths, pause_dur=40)        

    # # turning tensor into regular array
    # arr = torch.tensor_split(waveforms.squeeze(1), len(waveforms), dim=0)

    # # cut padding
    # arr = [a[:, :l] for a, l in zip(arr, mel_lengths * narrator.hop_len)]
    
    # mel_lengths = mel_lengths.detach().numpy()
    # batch_df["waveform"] = arr
    # batch_df["mel_lengths"] = mel_lengths
    # batch_df["durations_sec"] = mel_lengths / 22050.0
    # return batch_df


In [ ]:
# Apply the UDF
step1 = n_chunks.to_spark().sort("text_len", ascending=False ) 
step1.show()

In [ ]:
type(step1)

In [ ]:

step2 = step1.withColumn("Result", tts_udf("sentence")) 
step2.show()

In [ ]:
step3 = step2.select("index", "sentence", "text_len", "Result.*")


In [ ]:
@pandas_udf('string', PandasUDFType.SCALAR)
def tts_udf(batch):
    model = broadcast_model.value.to('cuda')
    memory_per_sentence = 500_000_000  # Estimate
    total_memory = torch.cuda.get_device_properties(0).total_memory * 0.8
    batch_size = int(total_memory / memory_per_sentence)

    results = []
    for i in range(0, len(batch), batch_size):
        batch = batch[i:i + batch_size]
        with torch.no_grad():
            inputs = [torch.tensor(sentence).to('cuda') for sentence in batch]
            outputs = model(inputs)
            results.extend(outputs.cpu().numpy())
    return results



In [ ]:


# Function to process sentences in a batch
def process_batch(partition):
    model = broadcast_model.value.to('cuda')
    results = []
    for sentence in partition:
        with torch.no_grad():
            output = model(sentence)
            results.append(output.cpu().numpy())
    return iter(results)

# Apply the function to each partition
processed_rdd = df.rdd.mapPartitions(process_batch)
result_df = processed_rdd.toDF(["processed_output"])

Set the executor's memory and GPU resources:

```
--conf spark.executor.memory=16G
--conf spark.executor.resource.gpu.amount=1
```

Adjust the number of partitions:
```
--conf spark.sql.shuffle.partitions=<num_partitions>
```



# End

# Trash 

In [ ]:
batch_df = chunker.get_test_batch(3,53)
batch_df


In [ ]:
batch_df.loc[:,"seq"] = batch_df.sentence.map(narrator.tts.text_to_seq)

In [ ]:
batch_df

In [ ]:
batch_df.seq.map(lambda s: len(s[0]))

In [ ]:
sentences = batch_df.sort_values("text_len" ,ascending = False).reset_index()
sentences

In [ ]:

narrator = Narrator()    
waveforms, mel_lengths = narrator.infer(sentences.sentence)


In [ ]:
waveforms.shape

In [ ]:
arr = torch.tensor_split(waveforms.squeeze(1), len(waveforms), dim=0)
arr = [a[0].squeeze(0).numpy() for a in arr]
# cut padding
#arr = [a[:, :l] for a, l in zip(arr, mel_lengths * narrator.hop_len)]

#mel_lengths = mel_lengths.detach().numpy()
# batch_df["waveform"] = arr
# batch_df["mel_lengths"] = mel_lengths
# batch_df["durations_sec"] = mel_lengths / 22050.0


In [ ]:
arr[0]

In [ ]:


output = pd.DataFrame({"waveform": arr, "mel_lengths": mel_lengths})
# sentences["waveforms"] = waveforms
# sentences["mel_lengths"] = mel_lengths    


In [ ]:
output

In [ ]:
output.shape

In [ ]:
output.loc[0,"waveform"]